In [ ]:
from __future__ import print_function, division     # Python 2/3 compatibility
from skimage import io                              # utilities to read and write images in various formats
import numpy as np                                  # array manipulation package
import matplotlib.pyplot as plt                      # plotting package
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 6)            # set default figure size
plt.rcParams['image.cmap'] = 'gray'                 # set default colormap to gray

# Assignment 5 : Image Compression

The following programming assignment involves the tasks related to image compression. We will be studying a lossy image compression technique based on discrete cosine transform (DCT) and a discrete fourier transform (DFT).

**Please, follow carefully the submission instructions given in the end of this notebook.** You are encouraged to seek information in other places than the course book and lecture material but remember to **list all your sources under references**.

If you experience problems that you cannot solve using the course material or the Python documentation, or have any questions regarding the programming assignments, please do not hesitate to contact the course assistant by sending an e-mail at dip@unioulu.oulu.fi. You can also join in for the Q & A session (schedule is given on the course page in Moodle) for this assignment.

**Please, fill in your personal details below.**

# Personal details:

* **Name(s) and student ID(s):** Matt Stirling 2208599
* **Contact information:** stirlingmattmarkus@gmail.com

# The CODEC in image compression pipelines
The pipeline for image compression has an encoder and a decoder. The encoder, typically has three blocks, viz., 1) Mapper, 2) Quantizer 3) Symbol coder. For more details, refer to the lecture notes or the Chapter 8 of the textbook. In brief,  <br />
    1) The mapper takes in your input image and transforms it into another form which is most easy to process. It removes *pixel or temporal redundancy* at this stage. This stage is reversible.<br />
    2) The quantizer typically reduces the *resolution* of the output of the mapper by removing the irrelevant information. A quantizer is an irreversible step, and leads to loss of information. **The tasks here revolve around this block primarily.** <br />
    3) The third stage is a symbol coder, which maps each output of the quantizer with a fixed-length or run-length code. *Typically, the shortest codes are mapped to the symbols which have highest probability of occurring, à la Huffman encoding.*

# 1. DCT-based block transform coding
The goal of this task, is to study the lossy compression techniques, specifically the DCT-based block transform coding. For this, we will be using the one-dimensional DCT available in scipy, [`dct`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.dct.html) and its inverse DCT available in scipy, [`idct`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.idct.html)

The goals of this task is to perform image compression using Discrete Cosine Transform. There are three main steps here. First is to compute a block DCT. Second step is to remove 20% of the smallest coefficients. Third step is to compute the block inverse DCT and reconstruct the image. 

**1.1 First read the test.jpg image and ensure that its datatype is float.**

In [ ]:
from scipy.fft import  idct, dct
# First read the grayscale test.jpg image. Make sure to convert the datatype to float
test = io.imread('test.jpg')
test = test.astype(float)

# Plot the given image as a grayscale
ax = plt.gca()
plt.imshow(test)
ax.axis('off')
plt.show()

**We will use the functions in scipy.fft package for computing DCT and inverse DCT transforms. However, they perform one-dimensional DCT and inverse DCT. Therefore, following is a wrapper to perform two-dimensional DCT and inverse DCT.**

In [ ]:
# Two functions are implemented for computing 2D DCT and 2D inverse DCT.
def dct2(a):
    return dct( dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return idct( idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho') 

**1.2 Next, use the given function for two-dimensional DCT to perform block DCT on the input image test.jpg. For this you need to block partition your image in blocks of size 8 x 8. </br> Compute the DCT for each 8 x 8 block and store it in a new numpy array. E.g., if your image has 32x32 pixels, then it will have 4 block rows and 4 block columns, each block being of 8x8 size.**

In [ ]:
# my own custom functions for partitioning and departitioning the 2d arrays into matrices of blocks. 
def partition_to_blocks(matrix, wid=8):
    (y_n, x_n) = (np.array(matrix.shape) / wid).astype(int)
    blocks = []
    for j in range(y_n):
        row = []
        for i in range(x_n):
            x, y = i*wid, j*wid
            sect = matrix[y:y+wid, x:x+wid]
            row.append(sect)
        blocks.append(row)
    return np.array(blocks)

def departition_blocks(matrix):
    rows = []
    for j in range(np.array(matrix).shape[0]):
        rows.append(np.concatenate(matrix[j], axis=1))
    full = np.concatenate(rows)
    return np.array(full)

In [ ]:
w = 8
blocks = partition_to_blocks(test, wid=w)

# Now go through each of the 8x8 blocks of the original image and compute its dct.
blocks_dct = np.copy(blocks)
for j in range(blocks.shape[0]):
    for i in range(blocks.shape[1]):
        blocks_dct[j][i] = dct2(blocks_dct[j][i])

# concatenate block matrix (couldn't figure out how to use r_)
full = departition_blocks(blocks)
test_dct = departition_blocks(blocks_dct)
        
# Display any one of the w x w sized blocks of the original image and its computed DCT block
i = 100 # Show four images, original image, its DCT, the 100-th block of the original image and the dct of this block.
(y_n, x_n) = (np.array(test.shape) / w).astype(int)
(x, y) = (i % x_n, int(i / y_n))

fig, ax = plt.subplots(2, 2, figsize=(8,8))

ax[0][0].imshow(test)
ax[0][0].set_title('original')
ax[0][0].axis('off')
ax[0][1].imshow(test_dct)
ax[0][1].set_title('dct')
ax[0][1].axis('off')
ax[1][0].imshow(blocks[y][x])
ax[1][0].set_title('block of original')
ax[1][0].axis('off')
ax[1][1].imshow(blocks_dct[y][x])
ax[1][1].set_title('dct of same block')
ax[1][1].axis('off')

fig.tight_layout()

**1.3 The next step is thresholding, i.e., pinning the DCT coefficients to be above a fraction of the maximum value. </br>
Set the threshold, to be 0.2, i.e., set thresh = 0.2. </br>Let max_dct to be the largest DCT coefficient value. Then keep all those DCT coefficients which are larger than 0.2*max_dct. </br> Also caluculate the % of DCT coefficients which are removed in thresholding.** 

In [ ]:
# Set the threshold
thresh = 0.2
max_dct = np.max(np.abs(test_dct))
#print("Max dct coefficient: ", max_dct)

# Select the dct coefficients which are larger than 20% of the maximum dct coefficient value.
test_dct_sel = np.copy(test_dct)

for j, row in enumerate(test_dct_sel):
    for i, coef in enumerate(row):
        if np.abs(coef) < thresh*max_dct:
            test_dct_sel[j][i] = 0.0

# Plot the original image, the computed DCTs and the selected DCTs using the threshold in the same image.
fig, ax = plt.subplots(1,3)

ax[0].imshow(test)
ax[0].set_title('original')
ax[0].axis('off')
ax[1].imshow(test_dct)
ax[1].set_title('dct')
ax[1].axis('off')
ax[2].imshow(test_dct_sel)
ax[2].set_title('dct selected coefficients')
ax[2].axis('off')

fig.tight_layout()

# Calculate the % of rejected dct coefficients above
rejected_coef_perc_dct = len(np.argwhere(test_dct_sel == 0.0)) / len(test_dct_sel.flatten()) * 100.0
print("Percentage of coefficients rejected: ", np.round(rejected_coef_perc_dct,2), "%")


**1.4 The next step is to reconstruct the compressed image from the thresholded DCT. Also, compute the RMSE error between the original image and the reconstructed image.**

In [ ]:
# Reconstruct the image using the two-dimensional inverse DCT wrapper given earlier here. 
blocks = partition_to_blocks(test_dct_sel, wid=8)
for j in range(blocks.shape[0]):
    for i in range(blocks.shape[1]):
        blocks[j][i] = idct2(blocks[j][i])

test_dct_reconstructed = departition_blocks(blocks)

# Display the original image and the reconstructed image in the same figure
fig, ax = plt.subplots(1,2)

ax[0].imshow(test)
ax[0].set_title('original')
ax[0].axis('off')
ax[1].imshow(test_dct_reconstructed)
ax[1].set_title('reconstructed')
ax[1].axis('off')

fig.tight_layout()

# print RMSE of the reconsutrcted image w.r.t. the original image. 
(h, w) = test_dct_reconstructed.shape
test_clipped = test[0:h, 0:w] # clip original to same shape as reconstructed
RMSE_dct = np.sqrt( np.mean( np.square(test_clipped - test_dct_reconstructed) ) )
print("Root mean square error of reconstructed image:", np.round(RMSE_dct,2))


**1.5 Instead of block DCT, now use block FFT to perform image compression. Use the same window as used for DCT. You can use the numpy functions  [`fft2`](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft2.html) and for inverse FFT, use the [`ifft2`](https://numpy.org/doc/stable/reference/generated/numpy.fft.ifft2.html)**

In [ ]:
from numpy.fft import fft2, ifft2

# Just as in the case of block DCT, here you need to perform block FFT. Use the same size for the block as done in 1.1 task. 
w = 8
blocks = partition_to_blocks(test)

# Now go through each of the 8x8 blocks of the original image and compute its fft.
blocks_fft = np.copy(blocks).astype('complex128')
for j in range(blocks.shape[0]):
    for i in range(blocks.shape[1]):
        blocks_fft[j][i] = fft2(blocks_fft[j][i])
test_fft = departition_blocks(blocks_fft)

# Select the fft coefficients which are larger than 20% of the maximum coefficient value.  
max_fft = np.max(np.abs(test_fft))
#print("Max fft coefficient: ", max_fft)

for j, row in enumerate(test_fft):
    for i, coef in enumerate(row):
        if np.abs(coef) < thresh*max_fft:
            test_fft[j][i] = 0.0

# Calculate the % of rejected fft coefficients
rejected_coef_perc_fft = len(np.argwhere(test_fft == 0.0)) / len(test_fft.flatten()) * 100
print("Percentage of rejected coefficients: ", np.round(rejected_coef_perc_fft, 2), "%")

# Compute the inverse fft for each of 8x8 blocks of the fft. This is the step of image reconstruction.
blocks = partition_to_blocks(test_fft)
for j in range(blocks.shape[0]):
    for i in range(blocks.shape[1]):
        blocks[j][i] = ifft2(blocks[j][i])
test_fft_reconstructed = departition_blocks(blocks)

# Remember that fft and ifft coefficients are complex. Therefore, for image display purpose, compute the absolute of the ifft coefficients.
# Display the original image, dct compressed image in 1.4 task and the fft compressed image obtained in the above step.
fig, ax = plt.subplots(1,3)

ax[0].imshow(test)
ax[0].set_title('original')
ax[0].axis('off')
ax[1].imshow(test_dct_reconstructed)
ax[1].set_title('dct compressed')
ax[1].axis('off')
ax[2].imshow(np.abs(test_fft_reconstructed))
ax[2].set_title('fft compressed')
ax[2].axis('off')

fig.tight_layout()

# print RMSE between the original image and the reconstructed image above.
(h, w) = test_fft_reconstructed.shape
test_clipped = test[0:h, 0:w] # clip original to same shape as reconstructed
RMSE_fft = np.sqrt( np.mean( np.square(test_clipped - np.abs(test_fft_reconstructed)) ) )
print("Root mean square error of reconstructed image:", np.round(RMSE_fft,2))


# Aftermath
Finally, fill your answers to the following questions:

**How much time did you need to complete this exercise?**

`About 3-4 hours`

**Did you experience any problems with the exercise? Was there enough help available? Should this notebook be more (or less) detailed?**

`I couldn't quite figure out how to apply the numpy r_[] thing. It took me a bit of testing to figure out a good method for 'partitioning' the matrices into blocks and departition them. Maybe there could have been more resources for that?`

`Also, it could be mentioned what quality of output was expected. My compressed images both removed around 98% of the coefficients, and had a severe loss in quality. I assume it was intentional to have them be so severely compressed. Perhaps there could have been a better way to chose the coefficients to eliminate, for example, I played around with using the log of the max to eliminate the effect of the maximum coefficients being outliers.`

# References
`https://medium.com/analytics-vidhya/what-are-dct-coefficients-and-how-jpeg-compression-works-7f46d1e22b4c`

# Submission

1. Before submitting your work, **check that your notebook (code) runs from scratch** and reproduces all the requested results by clicking on the menu `Kernel -> Restart & Run All`! Also, check that you have answered all the questions written in **bold**.
2. Clear all outputs and variables, etc. by click on the menu `Kernel -> Restart & Clear Output`. This may (or will) reduce the file size of your deliverable a lot! 
3. Rename this Jupyter notebook to **`DIP_PA5_[student number(s)].ipynb`** (e.g. `DIP_PA5_1234567.ipynb` if solo work or `DIP_PA5_1234567-7654321.ipynb` if pair work)
4. Send **only** the resulting Jupyter notebook (the file with extension `.ipynb`) to matteo.pedone@oulu.fi and use this notebook name as email subject (e.g. DIP_PA5_1234567) . Please **do not include your working folder or the test images** in your submission!